In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

In [2]:
data_test = pd.read_csv('./data/data_test.csv')

In [3]:
features = dd.read_csv('./data/features.csv', blocksize=25e6, sep='\t')

In [4]:
features_test = features.loc[features['id'].isin(data_test['id'])].compute()

### Обзор data_test

In [5]:
data_test['id'].shape

(71231,)

In [6]:
data_test['id'].nunique()

70152

In [7]:
non_unique = data_test['id'].shape[0] - data_test['id'].nunique()
non_unique

1079

In [8]:
data_test[data_test['id'].duplicated(keep=False)].sort_values(by='id').head()

,Unnamed: 0,id,vas_id,buy_time
552,552,274,4.0,1548018000
551,551,274,2.0,1547413200
10677,10677,4770,4.0,1547413200
10676,10676,4770,2.0,1548018000
20241,20241,8166,1.0,1546808400


Один и тот же пользователб может в разные моменты времени покупать и отказываться от разных услуг. Нужно использовать соответствующий профиль пользователя из featuresна момент поекпки или отказа.

### Обзор feature_test

In [9]:
features_test.duplicated().sum()

0

In [10]:
features_test.shape

(72552, 256)

In [11]:
features_test['id'].nunique()

70152

In [12]:
non_unique = features_test.shape[0] - features_test['id'].nunique()
non_unique

2400

In [13]:
features_test[features_test['id'].duplicated(keep=False)].sort_values(by='id').head()

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
4741,9755,1544,1535317200,56.090029,46.140888,126.899214,78.283202,121.481754,57.17821,-16.08618,...,-974.373846,-613.770792,-23.996269,-37.630448,-179.747724,-14.832889,-0.694428,-1.175933,-0.45614,0.0
2444,1995811,1544,1546203600,68.940029,11.930888,182.759214,415.013202,177.341754,393.90821,-16.08618,...,-883.373846,-541.770792,-22.996269,-37.630448,68.252276,-4.832889,-0.694428,-12.175933,-0.45614,1.0
4071,2167703,4279,1545598800,-34.499971,-142.449112,-45.900786,-192.516798,-51.318246,-213.62179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
3581,4265509,4279,1546203600,-35.399971,-143.349112,-49.340786,-193.416798,-54.758246,-214.52179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
4923,340551,10021,1546808400,-85.109971,-384.779112,-77.400786,-398.196798,-82.818246,-419.30179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0


In [14]:
dublicated = features_test[features_test['id'].duplicated(keep=False)].sort_values(by='id')['id'].value_counts()
dublicated

186365     2
506522     2
3840662    2
3912339    2
1825422    2
          ..
1748755    2
3605502    2
1291656    2
2010499    2
1994754    2
Name: id, Length: 2400, dtype: int64

### Merge data_test и features_test

In [15]:
data_merged_test = pd.merge(data_test, features_test, on='id')
data_merged_test.head()

,Unnamed: 0_x,id,vas_id,buy_time_x,Unnamed: 0_y,buy_time_y,0,1,2,3,...,243,244,245,246,247,248,249,250,251,252
0,0,3130519,2.0,1548018000,1871362,1536526800,-62.899971,-374.279112,-72.600786,-418.406798,...,-970.373846,-613.770792,-25.996269,-37.630448,-258.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,1,2000860,4.0,1548018000,2024591,1532293200,-96.799971,100.290888,-110.740786,140.903202,...,-977.373846,-613.770792,-25.996269,-37.630448,-254.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
2,2,1099444,2.0,1546808400,4041332,1541365200,-81.969971,-390.729112,-95.910786,-443.336798,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
3,3,1343255,5.0,1547413200,1945062,1536526800,259.130029,-52.249112,245.189214,-104.856798,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
4,4,1277040,2.0,1546808400,3503668,1532293200,331.170029,590.890888,317.229214,538.283202,...,-651.373846,-612.770792,1.003731,-36.630448,38.252276,-12.832889,-0.694428,-12.175933,12.54386,0.0


In [16]:
tmp = data_merged_test.loc[data_merged_test['id'].isin(dublicated.index)]
tmp.head()

,Unnamed: 0_x,id,vas_id,buy_time_x,Unnamed: 0_y,buy_time_y,0,1,2,3,...,243,244,245,246,247,248,249,250,251,252
118,118,1759413,6.0,1547413200,1010540,1540155600,-96.799971,-408.179112,-110.740786,-460.786798,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
119,118,1759413,6.0,1547413200,4211678,1531688400,-95.949971,-368.439112,-109.890786,-421.046798,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
123,122,2502506,2.0,1547413200,254525,1543179600,-96.799971,-270.469112,-110.740786,-323.076798,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
124,122,2502506,2.0,1547413200,2465416,1535317200,184.510029,-93.569112,170.569214,61.443202,...,-715.373846,-351.770792,-25.996269,-37.630448,-222.747724,-16.832889,-0.694428,-3.175933,-0.45614,0.0
139,137,3130584,2.0,1546808400,571842,1532898000,-96.799971,-103.099112,-110.740786,-92.146798,...,-876.373846,-587.770792,-24.996269,-36.630448,-203.747724,-14.832889,-0.694428,-1.175933,-0.45614,0.0


In [17]:
tmp['delta'] = abs(tmp['buy_time_x'] - tmp['buy_time_y'])
tmp.sort_values(by = ['Unnamed: 0_x', 'delta'], inplace=True)
tmp.head()

<ipython-input-17-51c25ed858b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['delta'] = abs(tmp['buy_time_x'] - tmp['buy_time_y'])
<ipython-input-17-51c25ed858b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.sort_values(by = ['Unnamed: 0_x', 'delta'], inplace=True)


,Unnamed: 0_x,id,vas_id,buy_time_x,Unnamed: 0_y,buy_time_y,0,1,2,3,...,244,245,246,247,248,249,250,251,252,delta
118,118,1759413,6.0,1547413200,1010540,1540155600,-96.799971,-408.179112,-110.740786,-460.786798,...,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,7257600
119,118,1759413,6.0,1547413200,4211678,1531688400,-95.949971,-368.439112,-109.890786,-421.046798,...,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,15724800
123,122,2502506,2.0,1547413200,254525,1543179600,-96.799971,-270.469112,-110.740786,-323.076798,...,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0,4233600
124,122,2502506,2.0,1547413200,2465416,1535317200,184.510029,-93.569112,170.569214,61.443202,...,-351.770792,-25.996269,-37.630448,-222.747724,-16.832889,-0.694428,-3.175933,-0.45614,0.0,12096000
139,137,3130584,2.0,1546808400,571842,1532898000,-96.799971,-103.099112,-110.740786,-92.146798,...,-587.770792,-24.996269,-36.630448,-203.747724,-14.832889,-0.694428,-1.175933,-0.45614,0.0,13910400


In [18]:
duplicates = tmp['Unnamed: 0_x'].duplicated()
duplicates_to_delete = duplicates[duplicates.values == True]
data_merged_test.drop(duplicates_to_delete.index, axis=0, inplace=True)
data_merged_test.head()

,Unnamed: 0_x,id,vas_id,buy_time_x,Unnamed: 0_y,buy_time_y,0,1,2,3,...,243,244,245,246,247,248,249,250,251,252
0,0,3130519,2.0,1548018000,1871362,1536526800,-62.899971,-374.279112,-72.600786,-418.406798,...,-970.373846,-613.770792,-25.996269,-37.630448,-258.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,1,2000860,4.0,1548018000,2024591,1532293200,-96.799971,100.290888,-110.740786,140.903202,...,-977.373846,-613.770792,-25.996269,-37.630448,-254.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
2,2,1099444,2.0,1546808400,4041332,1541365200,-81.969971,-390.729112,-95.910786,-443.336798,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
3,3,1343255,5.0,1547413200,1945062,1536526800,259.130029,-52.249112,245.189214,-104.856798,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
4,4,1277040,2.0,1546808400,3503668,1532293200,331.170029,590.890888,317.229214,538.283202,...,-651.373846,-612.770792,1.003731,-36.630448,38.252276,-12.832889,-0.694428,-12.175933,12.54386,0.0


In [19]:
data_merged_test.shape

(71231, 259)

In [20]:
data_merged_test.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'buy_time_y'], axis=1, inplace=True)
data_merged_test.head()

,id,vas_id,buy_time_x,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
0,3130519,2.0,1548018000,-62.899971,-374.279112,-72.600786,-418.406798,-78.018246,-439.51179,17.81382,...,-970.373846,-613.770792,-25.996269,-37.630448,-258.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,2000860,4.0,1548018000,-96.799971,100.290888,-110.740786,140.903202,-116.158246,119.79821,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-254.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
2,1099444,2.0,1546808400,-81.969971,-390.729112,-95.910786,-443.336798,-101.328246,-464.44179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
3,1343255,5.0,1547413200,259.130029,-52.249112,245.189214,-104.856798,239.771754,-125.96179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0
4,1277040,2.0,1546808400,331.170029,590.890888,317.229214,538.283202,311.811754,522.25821,-16.08618,...,-651.373846,-612.770792,1.003731,-36.630448,38.252276,-12.832889,-0.694428,-12.175933,12.54386,0.0


In [21]:
data_merged_test.to_csv('./data/my_test.csv')